In [159]:
import keras

In [160]:
'''
#Train a simple deep CNN on the CIFAR10 small images dataset.

It gets to 75% validation accuracy in 25 epochs, and 79% after 50 epochs.
(it's still underfitting at that point, though).
'''

from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils
import os

batch_size = 1000
num_classes = 10
epochs = 10
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)


x_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


In [161]:
print(x_train.shape[1:])
print(type(x_train))

(3, 32, 32)
<class 'numpy.ndarray'>


In [162]:
import numpy as np

In [163]:
def rgb2gray(rgb):
    r, g, b = rgb[:,0,:,:], rgb[:,1,:,:], rgb[:,2,:,:]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    gray = np.expand_dims(gray, axis=1)
    print(gray.shape)
    return gray

In [164]:
x_train=rgb2gray(x_train)
x_test=rgb2gray(x_test)

(50000, 1, 32, 32)
(10000, 1, 32, 32)


In [165]:
print(x_train[0].shape)

(1, 32, 32)


In [166]:
print(x_test.shape)

(10000, 1, 32, 32)


In [167]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [168]:
model = Sequential()
model.add(Convolution2D(2, 5, 5, border_mode='valid', input_shape=(1, 32, 32)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))   
model.add(Convolution2D(16, 5, 5, border_mode='valid'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(120))
model.add(Activation('relu'))
model.add(Dense(84))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

/usr/local/miniconda3/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(2, (5, 5), padding="valid", input_shape=(1, 32, 32...)`
  
/usr/local/miniconda3/lib/python3.5/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (5, 5), padding="valid")`
  """


In [169]:
print(x_train[0].shape)
print(x_train[0,:,:,:])

(1, 32, 32)
[[[  61.2111   44.9847   48.023  ...,  137.0222  130.4358  128.7672]
  [  18.8024    0.       10.0762 ...,   94.6907   89.9901   94.0328]
  [  23.9545    8.8914   31.4071 ...,   90.2782   90.876    80.2191]
  ..., 
  [ 172.9052  153.7659  156.6532 ...,  133.875    35.7334   38.0797]
  [ 146.339   128.6987  143.5434 ...,  152.5896   69.2633   59.7957]
  [ 150.6573  136.6542  146.7751 ...,  188.5304  123.9759   98.9767]]]


In [170]:
y_test.shape[1]

10

In [171]:
# Loss function
def neg_log_likelihood_loss(y_true, y_pred):
    ##sep = y_pred.shape[1] // 2
    mu, logvar = y_pred[:, :10], y_pred[:, 10:]
    return K.sum(0.5*(logvar+np.log(2*np.pi)+K.square((y_true-mu)/K.exp(0.5*logvar))), axis=-1)

In [172]:
# initiate RMSprop optimizer
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['categorical_crossentropy', 'accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [173]:
type(x_train)

numpy.ndarray

In [174]:
# model.fit(x_train, y_train,
#          batch_size=batch_size,
#          nb_epoch=epochs,
#          #validation_data=(x_test, y_test),
#          shuffle=True)

/usr/local/miniconda3/lib/python3.5/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
50000/50000 [==============================] - 0s - loss: 2.2437 - categorical_crossentropy: 2.2437 - acc: 0.1612     
Epoch 2/10
50000/50000 [==============================] - 0s - loss: 1.9468 - categorical_crossentropy: 1.9468 - acc: 0.3074     
Epoch 3/10
50000/50000 [==============================] - 0s - loss: 1.8006 - categorical_crossentropy: 1.8006 - acc: 0.3629     
Epoch 4/10
50000/50000 [==============================] - 0s - loss: 1.7161 - categorical_crossentropy: 1.7161 - acc: 0.3938     
Epoch 5/10
50000/50000 [==============================] - 0s - loss: 1.6368 - categorical_crossentropy: 1.6368 - acc: 0.4198     
Epoch 6/10
50000/50000 [==============================] - 0s - loss: 1.5957 - categorical_crossentropy: 1.5957 - acc: 0.4397     
Epoch 7/10
50000/50000 [==============================] - 0s - loss: 1.5307 - categorical_crossentropy: 1.5307 - acc: 0.4613     
Epoch 8/10
50000/50000 [==============================] - 0s - loss: 1.5008 - categorical_

In [176]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        nb_epoch=epochs,
                        validation_data=(x_test, y_test))

Using real-time data augmentation.
Epoch 1/10
50/50 [==============================] - 4s - loss: 1.5902 - categorical_crossentropy: 1.5902 - acc: 0.4423 - val_loss: 1.8923 - val_categorical_crossentropy: 1.8923 - val_acc: 0.3605
Epoch 2/10
50/50 [==============================] - 3s - loss: 1.5806 - categorical_crossentropy: 1.5806 - acc: 0.4462 - val_loss: 1.5020 - val_categorical_crossentropy: 1.5020 - val_acc: 0.4704
Epoch 3/10
50/50 [==============================] - 4s - loss: 1.5372 - categorical_crossentropy: 1.5372 - acc: 0.4608 - val_loss: 1.4260 - val_categorical_crossentropy: 1.4260 - val_acc: 0.4978
Epoch 4/10
50/50 [==============================] - 4s - loss: 1.5181 - categorical_crossentropy: 1.5181 - acc: 0.4644 - val_loss: 1.4149 - val_categorical_crossentropy: 1.4149 - val_acc: 0.5020
Epoch 5/10
50/50 [==============================] - 4s - loss: 1.5179 - categorical_crossentropy: 1.5179 - acc: 0.4671 - val_loss: 1.3963 - val_categorical_crossentropy: 1.3963 - val_ac

In [178]:
# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate_generator(datagen.flow(x_test, y_test,
                                      batch_size=batch_size),
                                      steps=x_test.shape[0] // batch_size)
print('Model Accuracy = %.2f, Model loss = %.2f ' % (evaluation[2], evaluation[1]))

Model Accuracy = 0.38, Model loss = 1.76 


In [175]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
##model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
#scores = model.evaluate(x_test, y_test,show_accuracy=True, verbose=1, batch_size=1000)
scores = model.evaluate(x_test, y_test, verbose=1, batch_size=2000)
print(scores)
#print('Test loss:', scores[0])
#print('Test accuracy:', scores[1])

Saved trained model at /root/shared/saved_models/keras_cifar10_trained_model.h5 
 2000/10000 [=====>........................] - ETA: 0s[1.5061332702636718, 1.5061332702636718, 0.47669999599456786]


In [77]:
print(batch_size)

64
